In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/prediction-interval-competition-ii-house-price/sample_submission.csv
/kaggle/input/prediction-interval-competition-ii-house-price/test.csv
/kaggle/input/prediction-interval-competition-ii-house-price/dataset.csv


In [2]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from pathlib import Path
from sklearn import preprocessing
from tqdm.notebook import tqdm
import lightgbm as lgb
import catboost as cb
import xgboost as xgb
from sklearn.base import clone
import joblib 
import warnings
warnings.filterwarnings("ignore")

In [3]:
N_FOLDS = 5
ALPHA = 0.10  # 90% coverage
HILL_CLIMB_STEPS = 100
MODEL_NAMES = ["LightGBM", "CatBoost", "XGBoost"]
INITIAL_WEIGHTS = [0.4, 0.3, 0.3]
N_JOBS = -1  # Use all available cores

In [4]:
train_path="/kaggle/input/prediction-interval-competition-ii-house-price/dataset.csv"
test_path="/kaggle/input/prediction-interval-competition-ii-house-price/test.csv"

train_df=pd.read_csv(train_path)
test = pd.read_csv(test_path)

train_df

,id,sale_date,sale_price,sale_nbr,sale_warning,join_status,join_year,latitude,longitude,area,...,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_lakesamm,view_otherwater,view_other,submarket
0,0,2014-11-15,236000,2.0,,nochg,2025,47.2917,-122.3658,53,...,0,0,0,0,0,0,0,0,0,I
1,1,1999-01-15,313300,NaN,26,nochg,2025,47.6531,-122.1996,74,...,0,0,0,0,0,1,0,0,0,Q
2,2,2006-08-15,341000,1.0,,nochg,2025,47.4733,-122.1901,30,...,0,0,0,0,0,0,0,0,0,K
3,3,1999-12-15,267000,1.0,,nochg,2025,47.4739,-122.3295,96,...,0,0,0,0,0,0,0,0,0,G
4,4,2018-07-15,1650000,2.0,,miss99,2025,47.7516,-122.1222,36,...,0,0,0,0,0,0,0,0,0,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,199995,2000-08-15,277500,1.0,,nochg,2025,47.5503,-122.0285,69,...,0,0,0,0,0,0,0,0,0,O
199996,199996,2019-07-15,1296000,2.0,,new,2025,47.6789,-122.1164,72,...,0,0,0,0,0,0,0,0,0,P
199997,199997,2018-06-15,845000,2.0,,new,2025,47.6939,-122.3542,6,...,0,0,0,0,0,0,0,0,0,B
199998,199998,2018-06-15,890000,2.0,,nochg,2025,47.6433,-122.0613,35,...,0,0,0,0,0,0,0,0,0,O


In [5]:
train_df = train_df.drop(columns=["submarket","subdivision","sale_nbr"])
test = test.drop(columns=["submarket","subdivision","sale_nbr"])
train_df

,id,sale_date,sale_price,sale_warning,join_status,join_year,latitude,longitude,area,city,...,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_lakesamm,view_otherwater,view_other
0,0,2014-11-15,236000,,nochg,2025,47.2917,-122.3658,53,FEDERAL WAY,...,0,0,0,0,0,0,0,0,0,0
1,1,1999-01-15,313300,26,nochg,2025,47.6531,-122.1996,74,KIRKLAND,...,0,0,0,0,0,0,1,0,0,0
2,2,2006-08-15,341000,,nochg,2025,47.4733,-122.1901,30,RENTON,...,0,0,0,0,0,0,0,0,0,0
3,3,1999-12-15,267000,,nochg,2025,47.4739,-122.3295,96,BURIEN,...,0,0,0,0,0,0,0,0,0,0
4,4,2018-07-15,1650000,,miss99,2025,47.7516,-122.1222,36,KING COUNTY,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,199995,2000-08-15,277500,,nochg,2025,47.5503,-122.0285,69,ISSAQUAH,...,0,0,0,0,0,0,0,0,0,0
199996,199996,2019-07-15,1296000,,new,2025,47.6789,-122.1164,72,REDMOND,...,0,0,0,0,0,0,0,0,0,0
199997,199997,2018-06-15,845000,,new,2025,47.6939,-122.3542,6,SEATTLE,...,0,0,0,0,0,0,0,0,0,0
199998,199998,2018-06-15,890000,,nochg,2025,47.6433,-122.0613,35,SAMMAMISH,...,0,0,0,0,0,0,0,0,0,0


In [6]:
def preprocess(df):
    # Convert date and extract features
    df['sale_date'] = pd.to_datetime(df['sale_date'])
    df['sale_year'] = df['sale_date'].dt.year
    df['sale_month'] = df['sale_date'].dt.month
    
    # Feature engineering
    df['age'] = df['sale_year'] - df['year_built']
    df['renovated'] = np.where(df['year_reno'] > 0, 1, 0)
    df['years_since_reno'] = np.where(df['renovated'], df['sale_year'] - df['year_reno'], 0)
    df['total_baths'] = df['bath_full'] + 0.75*df['bath_3qtr'] + 0.5*df['bath_half']
    df['total_value'] = df['land_val'] + df['imp_val']
    df['living_area'] = df['sqft'] + df['sqft_fbsmt']
    

    
    # Encode categoricals (simplified example)
    cat_cols = ['sale_warning', 'join_status', 'city', 'zoning']
    for col in cat_cols:
        df[col] = df[col].astype('category')

    
    return df.drop(columns=['sale_date', 'id'])

# Load and preprocess data
train_df = preprocess(train_df)
X_test = preprocess(test)

In [7]:
X = train_df.drop(columns=['sale_price'])
y = train_df['sale_price']

# Ordinal encoding.
cat_cols = ['sale_warning', 'join_status', 'city', 'zoning']
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
X[cat_cols] = encoder.fit_transform(X[cat_cols])
X_test[cat_cols] = encoder.transform(X_test[cat_cols])

In [8]:
def winkler_score(y_true, lower, upper, alpha=ALPHA):
    """Vectorized Mean Winkler Interval Score calculation"""
    width = upper - lower
    below = np.maximum(lower - y_true, 0)
    above = np.maximum(y_true - upper, 0)
    return width + (2/alpha) * (below + above)
    

def train_model(model, X_train, y_train, X_val):
    """Train model and predict on validation set"""
    model.fit(X_train, y_train)
    if isinstance(model, cb.CatBoostRegressor):
        preds = model.predict(X_val)
        means = preds[:, 0]
        stds = np.sqrt(preds[:, 1])
        lower = means - 1.645 * stds
        upper = means + 1.645 * stds
    else:
        lower = model["lower"].predict(X_val)
        upper = model["upper"].predict(X_val)
    return lower, upper

In [9]:
models = {
        "LightGBM": {
            "lower": lgb.LGBMRegressor(
                objective="quantile", 
                alpha=0.05,
                n_estimators=1500,
                learning_rate=0.05,
                num_leaves=63,
                subsample=0.8,
                subsample_freq=1,
                random_state=42
            ),
            "upper": lgb.LGBMRegressor(
                objective="quantile", 
                alpha=0.95,
                n_estimators=1500,
                learning_rate=0.05,
                num_leaves=63,
                subsample=0.8,
                subsample_freq=1,
                random_state=42
            )
        },
        "CatBoost": cb.CatBoostRegressor(
            loss_function="RMSEWithUncertainty",
            iterations=1500,
            learning_rate=0.05,
            depth=8,
            verbose=0,
            random_seed=42
        ),
        "XGBoost": {
            "lower": xgb.XGBRegressor(
                objective="reg:quantileerror", 
                quantile_alpha=0.05,
                tree_method="hist",
                n_estimators=1500,
                learning_rate=0.05,
                max_depth=8,
                subsample=0.8,
                random_state=42
            ),
            "upper": xgb.XGBRegressor(
                objective="reg:quantileerror", 
                quantile_alpha=0.95,
                tree_method="hist",
                n_estimators=1500,
                learning_rate=0.05,
                max_depth=8,
                subsample=0.8,
                random_state=42
            )
        }
    }

In [10]:
# Initialize storage for OOF predictions
oof_lowers = {model: np.zeros(len(X)) for model in MODEL_NAMES}
oof_uppers = {model: np.zeros(len(X)) for model in MODEL_NAMES}
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
    
print("Starting Cross-Validation...")
for fold, (train_idx, val_idx) in tqdm(enumerate(kf.split(X, y)), total=N_FOLDS, desc="Folds"):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # Train and predict for each model
    for model_name in MODEL_NAMES:
        if model_name == "CatBoost":
            model = clone(models["CatBoost"])
            model.fit(X_train, y_train)
            preds = model.predict(X_val)
            means = preds[:, 0]
            stds = np.sqrt(preds[:, 1])
            oof_lowers[model_name][val_idx] = means - 1.645 * stds
            oof_uppers[model_name][val_idx] = means + 1.645 * stds
                
        else:
                # Train lower quantile model
            lower_model = clone(models[model_name]["lower"])
            lower_model.fit(X_train, y_train)
            lower_pred = lower_model.predict(X_val)
                
                # Train upper quantile model
            upper_model = clone(models[model_name]["upper"])
            upper_model.fit(X_train, y_train)
            upper_pred = upper_model.predict(X_val)
                
            oof_lowers[model_name][val_idx] = lower_pred
            oof_uppers[model_name][val_idx] = upper_pred
    
    # Calculate MWIS for each model
model_scores = {}
print("\nModel Performance Evaluation:")
for model_name in MODEL_NAMES:
        # Ensure valid intervals
    lower = np.minimum(oof_lowers[model_name], oof_uppers[model_name])
    upper = np.maximum(oof_lowers[model_name], oof_uppers[model_name])
        
        # Calculate MWIS
    wis = winkler_score(y, lower, upper)
    mwis = np.mean(wis)
    model_scores[model_name] = mwis
        
        # Calculate coverage
    coverage = np.mean((y >= lower) & (y <= upper)) * 100
    print(f"{model_name}: MWIS = {mwis:.2f}, Coverage = {coverage:.2f}%")
    
    # Hill Climbing Optimization
print("\nStarting Hill Climbing Optimization...")
current_weights = np.array(INITIAL_WEIGHTS)
best_weights = current_weights.copy()
best_score = float('inf')
    
    # Calculate initial combined score
combined_lower = sum(w * oof_lowers[model] for w, model in zip(current_weights, MODEL_NAMES))
combined_upper = sum(w * oof_uppers[model] for w, model in zip(current_weights, MODEL_NAMES))
current_score = np.mean(winkler_score(y, combined_lower, combined_upper))
    
print(f"Initial MWIS: {current_score:.4f}")
    
    # Optimization loop
for step in tqdm(range(HILL_CLIMB_STEPS), desc="Hill Climbing"):
    improved = False
    candidate_weights = current_weights.copy()
        
        # Generate candidate weights
    perturbation = np.random.dirichlet([5] * len(MODEL_NAMES)) - 0.5/len(MODEL_NAMES)
    candidate_weights = candidate_weights + 0.1 * perturbation
    candidate_weights = np.maximum(candidate_weights, 0)
    candidate_weights /= candidate_weights.sum()
        
        # Calculate combined predictions
    combined_lower = sum(w * oof_lowers[model] for w, model in zip(candidate_weights, MODEL_NAMES))
    combined_upper = sum(w * oof_uppers[model] for w, model in zip(candidate_weights, MODEL_NAMES))
        
        # Calculate MWIS
    candidate_score = np.mean(winkler_score(y, combined_lower, combined_upper))
        
        # Update if improvement
    if candidate_score < best_score:
        best_score = candidate_score
        best_weights = candidate_weights.copy()
        current_weights = candidate_weights.copy()
        improved = True
        print(f"Step {step}: New best MWIS = {best_score:.4f}, Weights = {best_weights}")

Starting Cross-Validation...


Folds:   0%|          | 0/5 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.033273 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4041
[LightGBM] [Info] Number of data points in the train set: 160000, number of used features: 49
[LightGBM] [Info] Start training from score 185000.000000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029484 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4041
[LightGBM] [Info] Number of data points in the train set: 160000, number of used features: 49
[LightGBM] [Info] Start training from score 1435000.000000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029016 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory

Hill Climbing:   0%|          | 0/100 [00:00<?, ?it/s]

Step 0: New best MWIS = 328639.4706, Weights = [0.37907736 0.30752076 0.31340188]
Step 2: New best MWIS = 328401.5684, Weights = [0.37102598 0.32277452 0.30619949]
Step 5: New best MWIS = 328346.9837, Weights = [0.37573302 0.32349546 0.30077152]
Step 6: New best MWIS = 328334.7700, Weights = [0.37910108 0.32264494 0.29825398]
Step 7: New best MWIS = 328093.6545, Weights = [0.37300068 0.33846293 0.28853639]
Step 9: New best MWIS = 328045.4822, Weights = [0.3835561  0.33626607 0.28017784]
Step 13: New best MWIS = 328028.4018, Weights = [0.38353027 0.33724162 0.27922811]
Step 15: New best MWIS = 327885.8164, Weights = [0.39493788 0.34040034 0.26466177]
Step 16: New best MWIS = 327865.6910, Weights = [0.39345338 0.34221553 0.26433109]
Step 18: New best MWIS = 327806.0351, Weights = [0.376769   0.35366725 0.26956374]
Step 20: New best MWIS = 327613.1478, Weights = [0.36424658 0.37298711 0.26276632]
Step 22: New best MWIS = 327610.3695, Weights = [0.35366663 0.3795217  0.26681167]
Step 28: N

In [11]:
# Final model training
print("\nTraining Final Models...")
test_preds = {"lower": {}, "upper": {}}
    
for model_name in tqdm(MODEL_NAMES, desc="Models"):
    if model_name == "CatBoost":
        model = clone(models["CatBoost"])
        model.fit(X, y)
        preds = model.predict(X_test)
        means = preds[:, 0]
        stds = np.sqrt(preds[:, 1])
        test_preds["lower"][model_name] = means - 1.645 * stds
        test_preds["upper"][model_name] = means + 1.645 * stds
            
    else:
            # Train lower quantile model
        lower_model = clone(models[model_name]["lower"])
        lower_model.fit(X, y)
        test_preds["lower"][model_name] = lower_model.predict(X_test)
            
            # Train upper quantile model
        upper_model = clone(models[model_name]["upper"])
        upper_model.fit(X, y)
        test_preds["upper"][model_name] = upper_model.predict(X_test)
    
    # Combine test predictions
final_lower = sum(best_weights[i] * test_preds["lower"][model] 
                     for i, model in enumerate(MODEL_NAMES))
final_upper = sum(best_weights[i] * test_preds["upper"][model] 
                     for i, model in enumerate(MODEL_NAMES))
    
    # Ensure valid intervals
final_lower, final_upper = np.minimum(final_lower, final_upper), np.maximum(final_lower, final_upper)
final_lower = np.maximum(final_lower, 0)  # Ensure non-negative prices


Training Final Models...


Models:   0%|          | 0/3 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.036733 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4063
[LightGBM] [Info] Number of data points in the train set: 200000, number of used features: 49
[LightGBM] [Info] Start training from score 185000.000000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.035874 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4063
[LightGBM] [Info] Number of data points in the train set: 200000, number of used features: 49
[LightGBM] [Info] Start training from score 1434006.000000


In [12]:
    # Create submission
test_ids = test["id"]
submission = pd.DataFrame({
        "id": test_ids,
        "pi_lower": final_lower,
        "pi_upper": final_upper
    })
submission.to_csv("submission.csv", index=False)
print("Submission saved successfully")

submission = pd.read_csv("submission.csv")
submission.head(5)
    

Submission saved successfully


,id,pi_lower,pi_upper
0,200000,807402.306062,1.098688e+06
1,200001,570175.871717,7.571810e+05
2,200002,471484.314175,6.760720e+05
3,200003,310142.854146,4.376421e+05
4,200004,394523.095959,6.064315e+05


In [13]:
from rich.console import Console
from rich.table import Table

console = Console()

# Header
console.rule("[bold cyan]FINAL PERFORMANCE REPORT")

# Optimized Weights Table
weights_table = Table(title="Optimized Weights", title_style="bold green")
weights_table.add_column("Model", justify="left", style="cyan")
weights_table.add_column("Weight", justify="right", style="magenta")

for model, weight in zip(MODEL_NAMES, best_weights):
    weights_table.add_row(model, f"{weight:.4f}")

console.print(weights_table)

# Model Performance Table
scores_table = Table(title="Model MWIS Scores", title_style="bold green")
scores_table.add_column("Model", justify="left", style="cyan")
scores_table.add_column("MWIS Score", justify="right", style="yellow")

for model, score in model_scores.items():
    scores_table.add_row(model, f"{score:.4f}")

console.print(scores_table)

# Ensemble Score Highlighted
ensemble_table = Table(title="Ensemble MWIS Score", title_style="bold red")
ensemble_table.add_column("Model", style="bold", justify="left")
ensemble_table.add_column("MWIS Score", style="bold yellow", justify="right")
ensemble_table.add_row("Ensemble", f"{best_score:.4f}")

console.print(ensemble_table)

──────────────────────────────────────────── FINAL PERFORMANCE REPORT ─────────────────────────────────────────────

  Optimized Weights  
┏━━━━━━━━━━┳━━━━━━━━┓
┃ Model    ┃ Weight ┃
┡━━━━━━━━━━╇━━━━━━━━┩
│ LightGBM │ 0.4098 │
│ CatBoost │ 0.3936 │
│ XGBoost  │ 0.1966 │
└──────────┴────────┘

    Model MWIS Scores     
┏━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Model    ┃  MWIS Score ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ LightGBM │ 340537.3299 │
│ CatBoost │ 344540.1634 │
│ XGBoost  │ 348755.6951 │
└──────────┴─────────────┘

   Ensemble MWIS Score    
┏━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Model    ┃  MWIS Score ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ Ensemble │ 327030.4668 │
└──────────┴─────────────┘